# Artificial Neural Networks for waves decomposition (or discrete integral transform)

This project aims to use ANNs as a no-brainer for the decomposition of different types of waves, that is, waves described by several functions other than just sine and cosine.

In the case of complex exponentials, this can be seen as a way to obtain a Discrete Fourier Transform without the proper FFT methods. For Bessel functions, this would be equivalent to a Discrete Henkel Transform.

The approach to ANN trained from FFT I saw functionally is from [@endolith](https://gist.github.com/endolith/98863221204541bf017b6cae71cb0a89), from which I used part of the code.

This method is no way intended to be faster or more efficient than FFT, but more flexible and easy to expando to different types of waves.

As a disclaimer, i'm new at GitHub and these calculations are still being made, so if you find something incomplete, it will be done in due time.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [70]:
N=32; batch=10000;

In [96]:
pos=np.arange(N);A=np.random.randn(batch, N) + 1j*np.random.randn(batch, N);K=np.random.randint(0, N, size=(batch, N));x=np.arange(N);

In [97]:
expo=np.zeros((batch, N), dtype='complex128');
for j in range(batch):
    for i in pos:
        expo[j]+=A[j][i]*np.exp(K[j][i]*1j*2*np.pi*(x)/N)

In [98]:
ft=np.zeros_like(K, dtype='complex128');
for j in range(batch):
    for i in pos:
        ft[j][K[j][i]]+=(A[j][i])*N

In [99]:
X = np.hstack([expo.real, expo.imag])
Y = np.hstack([ft.real, ft.imag])

In [100]:
model = keras.Sequential([layers.Dense(N*2, input_dim=N*2, use_bias=False)])
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, Y, epochs=100, batch_size=100)

Train on 10000 samples
Epoch 1/100
10000/10000 [==============================] - 0s 48us/sample - loss: 959.4000
Epoch 2/100
10000/10000 [==============================] - 0s 18us/sample - loss: 785.1089
Epoch 3/100
10000/10000 [==============================] - 0s 18us/sample - loss: 636.7454
Epoch 4/100
10000/10000 [==============================] - 0s 18us/sample - loss: 511.6430
Epoch 5/100
10000/10000 [==============================] - 0s 18us/sample - loss: 407.0461
Epoch 6/100
10000/10000 [==============================] - 0s 17us/sample - loss: 320.4333
Epoch 7/100
10000/10000 [==============================] - 0s 17us/sample - loss: 249.4349
Epoch 8/100
10000/10000 [==============================] - 0s 17us/sample - loss: 191.8537
Epoch 9/100
10000/10000 [==============================] - 0s 17us/sample - loss: 145.7146
Epoch 10/100
10000/10000 [==============================] - 0s 17us/sample - loss: 109.1995
Epoch 11/100
10000/10000 [==============================] - 0s 17u

In [ ]:
data = np.arange(N)

In [ ]:
def ANN_DFT(x):
    if len(x) != N:
        raise ValueError(f'Input must be length {N}')
    pred = model.predict(np.hstack([x.real, x.imag])[np.newaxis])[0]
    result = pred[:N] + 1j*pred[N:]
    return result

In [ ]:
ANN = ANN_DFT(data)
FFT = np.fft.fft(data)
print(f'ANN matches FFT: {np.allclose(ANN, FFT)}')

In [ ]:
plt.plot(abs(ANN))
plt.show

In [ ]:
plt.plot(abs(FFT))
plt.show

In [ ]:
seno=[7*np.sin(3.5*2*np.pi*x/32) for x in data]

In [ ]:
sena=np.array(seno)
ANN = ANN_DFT(sena)
FFT = np.fft.fft(sena)
print(f'ANN matches FFT: {np.allclose(ANN, FFT, rtol=1e-05, atol=1e-05)}')